In [1]:
import numpy as np
import matplotlib as plt
from keras.models import Sequential, load_model
from PIL import Image
import timeit,time
import math
import pandas as pd

from skimage import measure
from scipy import ndimage as ndi
from skimage import feature

Using TensorFlow backend.


In [2]:
# load the model
path = '/home/grads/n/narendra5/Desktop/Programs/LER_machine_learning/'
#path = r"C:\\Users\\narendra\\Documents\\LER\\LER_machine_learning\\"

#model = load_model(path + r'models\\' + 'full_run1_epoch_4.h5')
model = load_model(path + 'models/' + 'full_run1_epoch_4.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1024, 64, 64)      640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024, 64, 64)      256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024, 64, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1024, 64, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 1024, 64, 64)      256       
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024, 64, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1024, 64, 64)      36928     
__________

In [6]:
"""
Entire Test set 

sigmas = [0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8]
Xis = [10, 30, 40]
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
widths = [20, 30]
noises = [2, 3, 4, 5, 10, 20, 30, 50, 100, 200]
"""

sigmas = [0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8]
Xis = [10, 30, 40]
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
widths = [20, 30]
noises = [2, 3, 4, 5, 10, 20, 30, 50, 100, 200]

testsize = len(sigmas)*len(Xis)*len(alphas)*len(widths)*len(noises)*2
print('Testsize: ', testsize)
#X_test = np.zeros([testsize, 1024, 64])
#y_test = np.zeros([testsize, 1024, 64])

df = pd.DataFrame(columns = ['noise', 'sigma', 'alpha', 'Xi', 'width', 'space', 'MSE_noise','MSE_pred', \
                             'PSNR_noise','PSNR_pred', 'Pred_time', 'O_leftline_sigma', 'i_leftline_sigma', \
                             'O_rightline_sigma', 'i_rightline_sigma', 'ledge_orig_sigma', 'ledge_pred_sigma', \
                             'redge_orig_sigma', 'redge_pred_sigma'])

count = 0
for sigma in sigmas:
    for alpha in alphas:
        for Xi in Xis:
            for width in widths:
                for s in range(2):
                    for noise in noises:
                        space = math.floor(width*2**s)
                        shift = math.floor(-25 + (width + space/2 + Xi + alpha*10 + sigma*10)%16) 
                        
                        original_file = path + 'original_images/oim_' + "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '.tiff'
                        noisy_file = path + 'noisy_images/nim_' + "{0:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '_' + str(noise) + '.tiff'
                        linescan_file = path + 'linescans/linescan_' + "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '.txt'
                        linescan = []
                        with open(linescan_file,'r') as f:
                            for line in f:
                                a, b = line.split(',')
                                linescan.append(float(b))
                        
                        linescan = linescan[:2048]
                        
                        leftline = np.array(linescan[:1024]) 
                        rightline = linescan[1024:]
                        rightline.reverse()
                        rightline = np.array(rightline)

                        leftline = leftline + shift
                        rightline = rightline + shift
                        
                        
                        im = np.array(Image.open(original_file))
                        imnoisy = np.array(Image.open(noisy_file))
                        
                        im = im/256
                        imnoisy = imnoisy/256
                        imnoisy = imnoisy.reshape(1,1024,64,1)
                        im = im.reshape(1,1024,64,1)
                        
                        #test_score = model.evaluate(imnoisy, im)
                        start = time.time()
                        impredict = model.predict(imnoisy)
                        prediction_time = time.time() - start
                        
                        
                        impredict = impredict.reshape(1024,64)
                        imnoisy = imnoisy.reshape(1024,64)
                        im = im.reshape(1024,64)
                        impredict = impredict.astype(float)
                        imnoisy = imnoisy.astype(float)
                        im = im.astype(float)
                        
                        mse_predict = (256**2)*((im - impredict) ** 2).mean()
                        mse_noisy = (256**2)*((im - imnoisy) ** 2).mean()
                        
                        psnr_noisy = measure.compare_psnr(im,imnoisy)
                        psnr_predict = measure.compare_psnr(im,impredict)
                        
                        canny_sigma = 1
                        edges_im = feature.canny(im, sigma = canny_sigma)
                        edges_imnoisy = feature.canny(imnoisy, sigma = canny_sigma)
                        edges_impredict = feature.canny(impredict, sigma = canny_sigma)
                        
                        ledge_orig = np.argmax(edges_im, axis = 1)[256:768]
                        redge_orig = np.argmax(np.fliplr(edges_im), axis = 1)[256:768]
                        
                        ledge_pred = np.argmax(edges_impredict, axis = 1)[256:768]
                        redge_pred = np.argmax(np.fliplr(edges_impredict), axis = 1)[256:768]
                        
                        df.loc[count] = [noise, sigma, alpha, Xi, width, space, mse_noisy, mse_predict, \
                                         psnr_noisy, psnr_predict, prediction_time, leftline[256:768].std()/2, \
                                         leftline[256:768].astype(int).std()/2, rightline[256:768].std()/2, \
                                         rightline[256:768].astype(int).std()/2, ledge_orig.std()/2, \
                                         ledge_pred.std()/2, redge_orig.std()/2, redge_pred.std()/2]
                        
                        count += 1
                        
print("Test count: ", count)
#print(df)

Testsize:  8640
Test count:  8640


In [7]:
#prediction_time = timeit.timeit(model.predict(imnoisy))

In [8]:
df.to_csv(path + 'Test_results.csv')

In [17]:
print(df.loc[(df['noise'] == 200) & (df['sigma'] == 1.2) & (df['alpha'] == 0.9)])

      noise  sigma  alpha    Xi  width  space  MSE_noise  MSE_pred  \
5289  200.0    1.2    0.9  10.0   20.0   20.0  80.299591  4.512715   
5299  200.0    1.2    0.9  10.0   20.0   40.0  81.604355  4.515404   
5309  200.0    1.2    0.9  10.0   30.0   30.0  88.340744  4.665813   
5319  200.0    1.2    0.9  10.0   30.0   60.0  88.845779  4.976666   
5329  200.0    1.2    0.9  30.0   20.0   20.0  80.491425  3.997804   
5339  200.0    1.2    0.9  30.0   20.0   40.0  79.693344  4.126473   
5349  200.0    1.2    0.9  30.0   30.0   30.0  87.964249  4.397863   
5359  200.0    1.2    0.9  30.0   30.0   60.0  86.165436  4.045603   
5369  200.0    1.2    0.9  40.0   20.0   20.0  81.469437  4.117485   
5379  200.0    1.2    0.9  40.0   20.0   40.0  78.442535  4.164296   
5389  200.0    1.2    0.9  40.0   30.0   30.0  88.701996  4.293029   
5399  200.0    1.2    0.9  40.0   30.0   60.0  86.704666  4.564992   

      PSNR_noise  PSNR_pred  Pred_time  O_leftline_sigma  i_leftline_sigma  \
5289   29.1